In [ ]:
import sys
sys.path.append("..") # append the directory above where the databese is.
import pandas as pd
import numpy as np
import sqlite3
import time
import numba
%load_ext line_profiler
### disable scientific notation in pandas
pd.set_option('display.float_format', '{:.2f}'.format) ### display up to 2 decimal pts

In [ ]:
connection = sqlite3.connect("../db.sqlite3")
cursor = connection.cursor() 
cursor.execute("SELECT * FROM CountryIndicators")  
country_list = cursor.fetchall() # we get the 
np.shape(country_list)
print(country_list[:3]) # okay it has the same format as the table where the rows are tuples in this list

In [ ]:
df = pd.DataFrame(country_list, columns=["CountryCode", "IndicatorCode", "Year", "Value" ])
df

In [ ]:
df[(df["IndicatorCode"] == "MS.MIL.XPRT.KD") & (df["Year"] == 2000) & df["CountryCode"] == "ARB"]

In [ ]:
df_ARB = df[df["CountryCode"] == "ARB"]
print(df_ARB["IndicatorCode"].nunique())
df_one_ind = df_ARB[df["IndicatorCode"] == "SP.ADO.TFRT"][["Year", "Value"]]
df_one_ind = df_one_ind.rename(columns={"Value":"indicator"})

df_another_ind = df_ARB[df["IndicatorCode"] == "MS.MIL.XPRT.KD"][["Year", "Value"]]
df_another_ind = df_another_ind.rename(columns={"Value": "another_indicator"})
pd.merge_ordered(df_one_ind, df_another_ind, left_by="Year", fill_method="ffill")
df[(df["CountryCode"] == "ARB") & (df["IndicatorCode"] == "SP.ADO.TFRT")][["Year", "Value"]].rename(columns={"Value":"SP.ADO.TFRT"})


In [ ]:
df_year = pd.DataFrame({"Year": df["Year"].unique()})
df_year

In [ ]:
df[df["CountryCode"] == "ARB"]["IndicatorCode"].nunique()

In [68]:
def create_final_df(df):
    dfs = []
    years = df["Year"].unique()
    for country in df["CountryCode"]:
        df_final = pd.DataFrame({"Year": years})
        df_country = df[df["CountryCode"] == country]
        for indicator in df_country["IndicatorCode"].unique():
            df_tmp = df_country[df_country["IndicatorCode"] == indicator][["CountryCode","Year", "Value"]]
            df_final = pd.merge_ordered(df_final, df_tmp.rename(columns={"Value": indicator}), fill_method="ffill")
        dfs.append(df_final)
        break
        pd.concat(dfs)
    return df_final

In [69]:
%lprun -f create_final_df df_final = create_final_df(df)

Timer unit: 1e-06 s

Total time: 94.867 s
File: <ipython-input-68-a9b5f19b62e8>
Function: create_final_df at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_final_df(df):
     2         1          3.0      3.0      0.0      dfs = []
     3         1      57442.0  57442.0      0.1      years = df["Year"].unique()
     4         1        117.0    117.0      0.0      for country in df["CountryCode"]:
     5         1       2515.0   2515.0      0.0          df_final = pd.DataFrame({"Year": years})
     6         1     357486.0 357486.0      0.4          df_country = df[df["CountryCode"] == country]
     7       630       4156.0      6.6      0.0          for indicator in df_country["IndicatorCode"].unique():
     8       629    2790463.0   4436.3      2.9              df_tmp = df_country[df_country["IndicatorCode"] == indicator][["CountryCode","Year", "Value"]]
     9       629   91654823.0 145715.1     96.6

In [70]:
df_final

,Year,CountryCode,SP.ADO.TFRT,SP.POP.DPND,SP.POP.DPND.OL,SP.POP.DPND.YG,MS.MIL.XPRT.KD,MS.MIL.MPRT.KD,SP.DYN.CBRT.IN,EN.ATM.CO2E.KT,...,SL.UEM.1524.NE.ZS,FB.BNK.CAPA.ZS,FB.CBK.DPTR.P3,IC.BUS.DFRN.XQ,DT.NFL.UNPB.CD,IC.ELC.TIME,SL.UEM.1524.FE.NE.ZS,SL.UEM.1524.MA.NE.ZS,EG.NSF.ACCS.RU.ZS,EG.NSF.ACCS.UR.ZS
0,1960,ARB,133.56,87.80,6.63,81.02,3000000.00,538000000.00,47.70,59563.99,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1961,ARB,134.16,89.22,6.75,82.32,0.00,578000000.00,47.46,65151.10,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1962,ARB,134.86,90.50,6.84,83.46,9000000.00,1414000000.00,47.20,74357.71,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,1963,ARB,134.50,91.60,6.92,84.41,8000000.00,1283000000.00,46.92,87895.98,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1964,ARB,134.10,92.42,6.97,85.10,2000000.00,1546000000.00,46.61,103196.28,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,1965,ARB,133.57,92.87,7.00,85.47,2000000.00,1945000000.00,46.27,123828.20,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,1966,ARB,132.68,93.73,7.07,86.25,13000000.00,3484000000.00,45.89,139079.15,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,1967,ARB,131.67,94.10,7.11,86.59,9000000.00,4753000000.00,45.48,149730.96,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,1968,ARB,129.20,94.12,7.12,86.63,35000000.00,3455000000.00,45.05,178805.04,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,1969,ARB,126.75,93.96,7.13,86.50,21000000.00,3860000000.00,44.61,213131.61,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:

df_ARB[df_ARB["IndicatorCode"] == "AG.AGR.TRAC.NO"]
#for indicator in df_ARB["IndicatorCode"]:
    #print(indicator)

In [ ]:
#def uniques(df, indicator):
    #""" returns true or false if the indicator is in every year"""
    #indicator_range = np.array(df[df["IndicatorCode"] == indicator]["Year"])
    #year_range = np.array(range(df["Year"].min(), df["Year"].max() + 1))
    #if len(year_range) != len(indicator_range):
     #   return False
    #return True
def create_total_df(df):
    df_final = pd.DataFrame(df["Year"])
   # print(df_final)
    #print(df["IndicatorCode"].nunique())
    for indicator in df["IndicatorCode"].unique():
        #print(f"{indicator}")
        #if uniques(df, indicator):
        df_tmp = df[df["IndicatorCode"] == indicator][["Year", "Value"]]
        df_tmp = df_tmp.rename(columns={"Value":indicator})
        df_final = pd.merge(df_final, df_tmp, "Year", how="outer")
        print(indicator)
    return df_final

#df_s = []
%lprun -f create_total_df create_total_df(df_ARB[df_ARB["Year"] == 2010])
#ncountrys = df["CountryCode"].nunique()
#count = 0
#for country in df["CountryCode"].unique():
 #   s = time.time()
  #  df_s.append(create_total_df(df[df["CountryCode"] == country]))
   # count += 1
    #print(country)
    #print(f"ETA: {(time.time() - s) *(ncountrys - 1)/60} min")


In [ ]:
def create_country_df(df_country):
    country_dict = {indicator:[] for indicator in df_country["IndicatorCode"].unique()}
    country_dict["Year"] = list(df_country["Year"].unique())
    for year in df_country["Year"].unique():
        for indicator in df_country["IndicatorCode"].unique():
            # this will output the indicator value for a given year
            df_tmp = df_country[(df_country["IndicatorCode"] == indicator) & (df_country["Year"] == year)]["Value"] 
            # if there is no indicator value for that year, then we add NA                              
            if df_tmp.empty:
                country_dict[indicator].append("NA")
            else:
                country_dict[indicator].append(df_tmp.to_list()[0]) # fix this is really ugly!!
        print(year)
    return pd.DataFrame(country_dict)
%lprun -f create_country_df create_country_df(df_ARB)

In [ ]:
def create_country_df(df_country):
    country_dict = {indicator:[] for indicator in df_country["IndicatorCode"].unique()}
    country_dict["Year"] = list(df_country["Year"].unique())
    for year in df_country["Year"].unique():
        for indicator in df_country["IndicatorCode"].unique():
            # this will output the indicator value for a given year
            df_tmp = df_country[(df_country["IndicatorCode"] == indicator) & (df_country["Year"] == year)]["Value"] 
            # if there is no indicator value for that year, then we add NA                              
            if df_tmp.empty:
                country_dict[indicator].append("NA")
            else:
                country_dict[indicator].append(df_tmp.to_list()[0]) # fix this is really ugly!!
        print(year)
    return pd.DataFrame(country_dict)